In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libraries and packages 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import uniform, randint

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedShuffleSplit, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import auc, accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import xgboost as xgb
import seaborn as sns

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_validate
from numpy import mean

In [ ]:
df = pd.read_csv("../input/credit-risk-dataset/credit_risk_dataset.csv")

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df = df[df["person_age"] < 90]
df = df[df["person_emp_length"] < 120]

In [ ]:
class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, columns, factor=1.5):
        self.columns = columns
        self.factor = factor
        
    def outliers_iqr(self, X, y=None):
        for col in self.columns:
            X = X[(X[col] >= self.lower_bound[col]) & (X[col] <= self.upper_bound[col])]
        return X
    
    def fit(self, X, y=None):
        self.y = y
        q1 = X[self.columns].quantile(0.25)
        q3 = X[self.columns].quantile(0.75)
        iqr = q3 - q1
        self.lower_bound = q1 - (iqr * self.factor)
        self.upper_bound = q3 + (iqr * self.factor)
        return self
    
    def transform(self, X, y=None):
        return self.outliers_iqr(X)

In [ ]:
features = df.drop(columns=["loan_status"], axis=1)
labels = df["loan_status"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.15, random_state=42)

In [ ]:
categorical = list(features.select_dtypes(include="object"))
numerical = list(features.select_dtypes(exclude="object"))

categorical, numerical

In [ ]:
class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, columns, factor=1.5):
        self.columns = columns
        self.factor = factor
        
    def fit(self, X, y=None):
        q1 = X[self.columns].quantile(0.25)
        q3 = X[self.columns].quantile(0.75)
        iqr = q3 - q1
        self.lower_bound = q1 - (iqr * self.factor)
        self.upper_bound = q3 + (iqr * self.factor)
        
        return self
    
    def transform(self, X, y=None):
        X = pd.DataFrame(X).copy()
        for c in self.columns:
            x = X.loc[:, c].copy()
            x[(x < self.lower_bound[c]) | (x > self.upper_bound[c])] = np.nan
            X.loc[:, c] = x
        return X

In [ ]:
cat_pipe = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown='ignore', sparse=False))
#     ("encoder", OrdinalEncoder())
])

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler()),
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipe, categorical),
    ("num", num_pipe, numerical)
])

In [ ]:
x_train

In [ ]:
# cat_columns = preprocessor.named_transformers_['cat']['encoder'].get_feature_names(categorical)
# columns = np.append(cat_columns, numerical)

# display(pd.DataFrame(preprocessor.transform(x_train), columns=columns))
preprocessor.transform(x_train)

In [ ]:
y_train.value_counts() / len(y_train), y_test.value_counts() / len(y_test)

In [ ]:
pipe = Pipeline([
#     ("smote", SMOTENC(categorical_features=[df.dtypes==object], random_state=0)),
#     ('outlier_removal', OutlierRemover(columns = likely_to_have_outlier)),
    ('preprocessor', preprocessor),
#     ("smote", SMOTE(sampling_strategy="minority", random_state=0)),
#     ('model', LogisticRegression(solver="liblinear", class_weight="balanced", penalty="l1")),
#     ('reduce_dim', PCA(n_components=15)),
#     ('model', LogisticRegression(solver="liblinear", class_weight={0: 0.5, 1: 1.5}, penalty="l1")),
#     ('model', RandomForestClassifier(class_weight={0: 0.5, 1: 2})),
    ('model', SVC(kernel="rbf", class_weight="balanced", tol=1e-5, C=100, gamma=5)),
])

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
scores = cross_validate(pipe, x_train, y_train, scoring='average_precision', cv=cv, n_jobs=-1)
scores

In [ ]:
pipe.fit(x_train, y_train)

In [ ]:
y_pred = pipe.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
threshold = np.linspace(0.5, 0.95, 20)
threshold

In [ ]:
def predict_with_threshold(model, x_test, y_test, threshold):
    y_pred = model.predict_proba(x_test)[:, 1] >= threshold
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [ ]:
for t in threshold:
    print("Threshold value:" + str(t))
    predict_with_threshold(pipe, x_test, y_test, t)

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
y_train.name

In [ ]:
print(auc(y_test, y_pred))

In [ ]:
pipe_logistic_reg = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(class_weight="balanced", tol=1e-5)),
])

params_logistic_grid = [
    {"model__solver": ["liblinear"], "model__penalty": ["l1", "l2"], "model__C": [1e-2, 1e-1, 1, 10, 100]},
    {"model__solver": ["newton-cg", "sag"], "model__penalty": ["none", "l2"], "model__C": [1e-2, 1e-1, 1, 10, 100]},
    {"model__solver": ["saga"], "model__penalty": ["none", "l1", "l2", "elasticnet"], "model__C": [1e-2, 1e-1, 1, 10, 100]},
]
# param_logistic_grid = {
#     "model__solver": ['liblinear', 'saga'],
#     "model__penalty": ['l1', 'l2'],
#     "model__C": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
# }

search_logistic_reg = GridSearchCV(pipe_logistic_reg, param_logistic_grid, n_jobs=-1)

In [ ]:
search_logistic_reg.fit(x_train, y_train)

In [ ]:
y_pred = search_logistic_reg.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
dump(search_logistic_reg, "logistic_regression.joblib")

In [ ]:
y_pred = search_logistic_reg.best_estimator_.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
pipe_randomforest = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(class_weight={0: 1, 1:2})),
])

pipe_randomforest.fit(x_train, y_train)
pipe_randomforest.par

In [ ]:
[estimator.tree_.max_depth for estimator in pipe_randomforest["model"].estimators_]

In [ ]:
pipe_randomforest["model"]

In [ ]:
pipe_randomforest = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(class_weight="balanced")),
])

params_randomforest_grid = {
    "model__bootstrap": [True, False],
    'model__max_depth': [15, 25, 35, None],
    'model__max_features': [None, "sqrt"],
    'model__n_estimators': [50, 100, 150],
    "model__criterion": ["gini", "entropy"]
}
search_randomforest = GridSearchCV(pipe_randomforest, params_randomforest_grid, n_jobs=-1)

In [ ]:
search_randomforest.fit(x_train, y_train)

In [ ]:
y_pred = search_randomforest.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
y_pred = search_randomforest.best_estimator_.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
search_randomforest.best_estimator_

In [ ]:
from joblib import dump, load

In [ ]:
!pwd

In [ ]:
dump(search_randomforest, 'random_forest.joblib')

In [ ]:
pipe_svm = Pipeline([
    ("preprocessor", preprocessor),
    ("model", SVC(class_weight="balanced")),
    
])

param_svm_grid = [
    {"model__kernel": ["rbf"], "model__gamma": [1e-3, 1e-4], "model__C": [1, 10, 100, 1000]},
    {"model__kernel": ["linear"], "model__C": [1, 10, 100, 1000]},
    {"model__kernel": ["poly"], "model__C": [1, 10, 100, 1000], "model__degree": [2, 3, 4]}
]

search_svm = GridSearchCV(pipe_svm, param_svm_grid, n_jobs=-1)

In [ ]:
search_svm.fit(x_train, y_train)

In [ ]:
y_pred = search_svm.best_estimator_.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
dump(search_svm, 'svm.joblib')

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
def convert_2_tf(x, y):
    

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.int64)
    
    return x, y

# def create_dataset(xs, ys, n_classes=10):
#     preprocessor.fit(xs)
#     xs = preprocessor.transform(xs)
#     ys = tf.one_hot(ys, depth=n_classes)
#     return tf.data.Dataset.from_tensor_slices((xs, ys)) \
#         .map(preprocess) \
#         .shuffle(len(ys)) \
#         .batch(128)

def create_tf_dataset(x, y, num_classes=10):
    preprocessor.fit(x)
    x = preprocessor.transform(x)
    y = tf.one_hot(y, depth=num_classes)
    return tf.data.Dataset.from_tensor_slices((x, y)) \
        .map(preprocess) \
        .shuffle(len(y)) \
        .batch(128)

train_dataset = create_tf_dataset(x_train, y_train, num_classes=2)
test_dataset = create_tf_dataset(x_test, y_test, num_classes=2)

In [ ]:
train_dataset

In [ ]:
mlp_model = keras.Sequential([
    keras.layers.Dense(units=64, activation="relu"),
    keras.layers.Dense(units=128, activation="relu"),
    keras.layers.Dense(units=2, activation='sigmoid')
])

In [ ]:
len(train_dataset)

In [ ]:
mlp_model.compile(optimizer='adam', 
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=METRICS)

history = mlp_model.fit(
    train_dataset.repeat(), 
    epochs=25, 
    steps_per_epoch=250,
    validation_data=test_dataset.repeat(), 
    validation_steps=2,
)

In [ ]:
y_pred = np.argmax(mlp_model.predict(test_dataset), axis=1)

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))